# Using Amazon SageMaker Object Detection For Playing Cards

# Table of Contents
1. [Setup](#Setup)
2. [Data Exploration](#Data-Exploration)

## Setup

Before inspecting and understanding the data, there are some initial steps to prepare the underlying notebook instance with additional Python libraries.

* **jsonlines** is used for easy interaction with JSON records stored as lines in a file. In this workshop, we use a SageMaker object detection [Augmented Manifest](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html#object-detection-augmented-manifest-training) file, allowing SageMaker to stream training data into the training job using Pipe Input mode.

In [ ]:
import sys
!{sys.executable} -m pip install jsonlines

Set up the S3 bucket where the training data is stored:

In [ ]:
bucket_training = 'remars2019-revegas-trainingdata'

To train the Object Detection algorithm on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To begin with, we need an AWS account role with SageMaker access. Here we will use the execution role the current notebook instance was given when it was created. This role has necessary permissions, including access to your data in S3.

We also import other libraries we need for the rest of the workshop to keep things organized.

In [ ]:
%%time
%matplotlib inline
import sagemaker
from sagemaker import get_execution_role
import boto3
s3 = boto3.resource('s3')
import json
import jsonlines
import random
import pandas as pd
from pandas.io.json import json_normalize
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import io

role = get_execution_role()
print(role)
sess = sagemaker.Session()

In [ ]:
plt.rcParams['figure.figsize'] = [15.5, 6]

## Data Exploration

Let's begin by inspecting the annotated (labelled) data provided for the workshop.

1. First, we'll download the Augmented Manifest file, which is a text file storing JSON objects on new lines. It stores references to image locations in S3, as well as the corresponding labels of suit and rank.

In [ ]:
s3.Bucket(bucket_training).download_file(
    'manifests/augmentedManifest.json',
    './full_manifest.json')

2. We'll visualize the training data distribution and make sure our classes are balanced. SageMaker uses a key-value map to pass labels as classes to a neural network model. We defined those statically below:

In [ ]:
class_map = {"AC": 0, "2C": 1, "3C": 2, "4C": 3, "5C": 4, "6C": 5, "7C": 6, "8C": 7, "9C": 8, "10C": 9, "JC": 10, 
             "QC": 11,"KC": 12, "AD": 13, "2D": 14, "3D": 15, "4D": 16, "5D": 17, "6D": 18, "7D": 19, "8D": 20, 
             "9D": 21, "10D": 22, "JD":23, "QD": 24, "KD": 25, "AH": 26, "2H": 27, "3H": 28, "4H": 29, "5H": 30, 
             "6H": 31, "7H": 32, "8H": 33, "9H": 34, "10H": 35, "JH": 36, "QH": 37, "KH": 38, "AS": 39, "2S": 40, 
             "3S": 41, "4S": 42, "5S": 43, "6S": 44, "7S": 45, "8S": 46, "9S": 47, "10S": 48, "JS": 49, "QS": 50, "KS": 51}
object_categories = list(class_map.keys())

In [ ]:
def get_key(val): 
    for key, value in class_map.items(): 
         if val == value: 
             return key 
    return "key doesn't exist"

labels = []
train_df = pd.read_json('full_manifest.json', lines=True)
# train_df['bounding-box'].values
for x in train_df['bounding-box'].values:
    for n in x['annotations']:
        # print(get_key(n['class_id']))
        labels.append(get_key(n['class_id']))
        
labels, values = zip(*Counter(labels).items())

indexes = np.arange(len(labels))
width = 1

plt.bar(indexes, values, width)
plt.xticks(indexes + width * 0.5, labels, rotation=45)
plt.show()

3. Let's randomly display a training image:

In [ ]:
whole_manifest = []

with jsonlines.open('full_manifest.json') as reader:
    for obj in reader:
        whole_manifest.append(obj)

In [ ]:
def display_train_img(whole_manifest):
    random_annotation = whole_manifest[random.randint(0,4999)]
    s3_uri = random_annotation['source-ref']
   
    s3_key = os.path.basename(s3_uri)
    if 'images' not in os.listdir('.'):
        os.mkdir('./images')
    s3.Bucket(bucket_training).download_file(
    s3_key, './images/' + s3_key)
    raw_img = mpimg.imread('images/' + s3_key)
    plt.imshow(raw_img)

display_train_img(whole_manifest)

As you can see, we have synthetically generated a training set of 5000 images using data augmentation techniques. By copying a cropped playing card onto various backgrounds and applying image filters such as blur and jpeg compression, the model should be much more robust.

## Data Preparation

Now that we have inspected the data, let's perform a few steps to get the data ready to train on Amazon SageMaker.

### Create train, validation, and test splits

In [ ]:
random.shuffle(whole_manifest)

count_samples = 0
for x in whole_manifest:
    count_samples = count_samples+1
    
print("Total samples: {}".format(count_samples))

train_count = round(count_samples * 0.04)
val_count = round(count_samples * 0.004)
test_count = round(count_samples * 0.002)
print("Train count: " + str(train_count) + '\n' +\
      "Validation count: " + str(val_count)  + '\n' +\
      "Test count: " + str(test_count))

In [ ]:
train_manifest = []
for i in range(train_count):
    train_manifest.append(whole_manifest.pop())
    
val_manifest = []
for i in range(val_count):
    val_manifest.append(whole_manifest.pop())
    
test_manifest = []
for i in range(test_count):
    test_manifest.append(whole_manifest.pop())

In [ ]:
with jsonlines.open('train.manifest', mode='w') as writer:
    for i in train_manifest:
        writer.write(i)
        
with jsonlines.open('validate.manifest', mode='w') as writer:
    for i in val_manifest:
        writer.write(i)
        
with jsonlines.open('test.manifest', mode='w') as writer:
    for i in test_manifest:
        writer.write(i)

### Upload the manifests to a location in S3 to be used in the training job:

In [ ]:
sess.upload_data(path='train.manifest', key_prefix='manifests')
sess.upload_data(path='validate.manifest', key_prefix='manifests')

s3_train_data = 's3://{}/manifests/{}'.format(sess.default_bucket(), 'train.manifest')
s3_validation_data = 's3://{}/manifests/{}'.format(sess.default_bucket(), 'validate.manifest')

## Train the model

In the following steps, you will incrementally train a model that we trained in advance over hundreds of thousands of images.

In [ ]:
s3_output_path = 's3://{}/card-detection-output/'.format(sess.default_bucket())

# Model URI to our previously trained model:
model_uri = 's3://remars2019-revegas-trainingdata/model.tar.gz'

# Training container image that has the built-in SageMaker algorithm:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, 'object-detection', repo_version='latest')

# Create the sagemaker estimator object.
playing_card_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count = 1, 
                                         train_instance_type = 'ml.p2.xlarge',
                                         input_mode='Pipe',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         output_path = s3_output_path,
                                         base_job_name = 'playingcard-bbox',
                                         sagemaker_session = sess,
                                         model_uri=model_uri)

In [ ]:
# Setup hyperparameters 
playing_card_model.set_hyperparameters(base_network='resnet-50',
                             kv_store='dist_sync',
                             mini_batch_size=16,
                             use_pretrained_model=1,                          
                             num_classes=52, # suit/rank combinations
                             epochs=30,
                             image_shape=512,
                             num_training_samples = train_count,
                             learning_rate=0.00001,                             
                             optimizer='sgd',
                             early_stopping=False,
                             lr_scheduler_factor=0.1,
                             lr_scheduler_step='20,25')

In [ ]:
# Create sagemaker s3_input objects

attribute_names = ["source-ref","bounding-box"]
distribution = 'FullyReplicated'

train_data = sagemaker.session.s3_input(s3_train_data, distribution=distribution, 
                                        content_type='application/x-recordio',
                                        record_wrapping='RecordIO',
                                        attribute_names=attribute_names,
                                        s3_data_type='AugmentedManifestFile')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution=distribution, 
                                        content_type='application/x-recordio',
                                        record_wrapping='RecordIO',
                                        attribute_names=attribute_names,
                                        s3_data_type='AugmentedManifestFile')

data_channels = {'train': train_data, 
                 'validation': validation_data}

In [ ]:
%%time
playing_card_model.fit(inputs=data_channels, logs=True)

## Deploy the model predictor

In [ ]:
pcm_predictor = playing_card_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
pcm_predictor.content_type('image/jpeg')

In [ ]:
def display_test_img(test_manifest):
    random_annotation = test_manifest[random.randint(0,test_count-1)]
    s3_uri = random_annotation['source-ref']
    annotations = random_annotation['bounding-box']['annotations']
    
    s3_key = os.path.basename(s3_uri)
    s3.Bucket(bucket_training).download_file(
    s3_key, 'images/' + s3_key)
    raw_img = mpimg.imread('images/' + s3_key)
    plt.imshow(raw_img)
    return s3_key

print(s3_key)
s3_key = display_test_img(test_manifest)

In [ ]:
def generate_predictions(s3_key):
    img_bytes = io.BytesIO()
    s3.Object(bucket_training, s3_key).download_fileobj(img_bytes)
    
    dets = json.loads(pcm_predictor.predict(img_bytes.getvalue()))
    return dets['prediction'], img_bytes

def visualize_detection(img_file, dets, classes=[], thresh=0.4):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """

        img = mpimg.imread(img_file, "jpg")
        plt.imshow(img)
        height = img.shape[0]
        width  = img.shape[1]
        colors = dict()
        num_detections = 0
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            num_detections += 1
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id], linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            print('{},{}'.format(class_name,score))
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')

        print('Number of detections: ' + str(num_detections))
        plt.show()

In [ ]:
detections, img = generate_predictions(s3_key=s3_key)

visualize_detection(img_file=img, dets=detections, classes=object_categories)

Make the model file accessible so that workshop leads can add your model to the blackjack table system!

In [ ]:
from urllib.parse import urlparse

print(playing_card_model.model_data)
o = urlparse(playing_card_model.model_data)

s3object = s3.Object(o.netloc,o.path.lstrip('/'))

print(o.netloc)
print(o.path.lstrip('/'))
                     

s3object.copy_from(
    ACL="public-read",
    CopySource={"Bucket": o.netloc,
                "Key": o.path.lstrip('/')
    }
)

In [ ]:
pcm_predictor.delete_endpoint()